# Quantifying tropical deforestation driven by oil palm cultivation in Borneo Island, Indonesia

## 0. Import libraries

In [1]:
import glob, os, time
import numpy as np
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
import shutil
import matplotlib.pyplot as plt
import sklearn
from pathlib import Path
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, classification_report
from rasterio import features
from scipy.ndimage import median_filter




ModuleNotFoundError: No module named 'rasterio'

## 1. Download of monthly composite from GEE

In [1]:
// Define start and end dates for the loop
//change start and end year get the monthly products of those years dowmloaded in google drive
var startYear = 2017;
var endYear = 2021;

// create a loop that iterates through each year and each month within that year.   

// Loop through each year
for (var year=startYear; year<=endYear; year++) {
  // Loop through each month
  for (var month=1; month<=12; month++) {
    
    // Define start and end dates for the current month
    var startDate = ee.Date.fromYMD(year, month, 1);
    var endDate = startDate.advance(1, 'month');
    
    // Filter the image collection by date and clip to the ROI
    var collectionVH = ee.ImageCollection('COPERNICUS/S1_GRD')
      .filter(ee.Filter.eq('instrumentMode', 'IW'))
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) //select VV for downloading VV
      .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
      .filterBounds(roi)
      .select(['VH'])                                                          //select VV for downloading VV 
      .filterDate(startDate, endDate);
    
    // Check if the collection is not empty
    var count = collectionVH.size().getInfo();
    if (count === 0) {
      print('No images found for ' + startDate.format('YYYY-MM-dd') + ' to ' + endDate.format('YYYY-MM-dd'));
      continue;
    }
    
    // Create a monthly median composite and clip to the ROI
    var monthlyComposite = collectionVH.median().clip(roi);
    
    // Define the export parameters
    var exportName = ee.String('s1_VH_median_').cat(startDate.format('YYYY-MM')).cat('_cliped');
    var exportFolder = 'LBRAT2104/poster';
    var exportScale = 10;

    // Export the image to Google Drive
    Export.image.toDrive({
      image: monthlyComposite,
      description: exportName.getInfo(),
      folder: exportFolder,
      scale: exportScale,
      maxPixels: 1e13,
      region: roi
    });
    
  }
}

## 2. Normalization 

The first step is to load the images (here VH but it must also be done for VV).

In [1]:
# Set the directory path
dir_path = 'X:\STUDENTS\GROUP_10\S1 images\VH'

# Get the list of file names in the directory
file_names = os.listdir(dir_path)

# Initialize an empty list to store the images
images = []

# Loop over the file names and load each image into a NumPy array
for file_name in file_names:
        # Load the image into a NumPy array
        with rasterio.open(os.path.join(dir_path, file_name)) as dataset:
            image = dataset.read(1)
        # Append the image to the list of images
        images.append(image)       

print(file_names)

To normalize an image, the following formula is used:

$image_{normalized} = (image - median_ {2years})/St. Dev. _{2years}$



### Calculation of the median for all images

In [ ]:
# Convert the list of images to a NumPy array
images_array = np.array(images)

# Initialize an empty list to store the median images
median_images = []

# Loop over the images and compute the median of each group of 24 images
for i in range(1, len(images_array) - 23):
    group = images_array[i-1:i+23]
    median_image = np.median(group, axis=0)
    median_images.append(median_image)

# Stack the median images into a multi-dimensional array
stacked_median_images = np.stack(median_images)

# Set the output file name
output_file = 'stacked_median_images.npy'

# Save the stacked_median_images array as a numpy file
np.save(output_file, stacked_median_images)

### Calculation of the standard deviation for all images

In [ ]:

# Convert the list of images to a NumPy array
images_array = np.array(images)

# Initialize an empty list to store the std images
std_images = []

# Loop over the images and compute the std of each group of 24 images
for i in range(1, len(images_array) - 23):
    group = images_array[i-1:i+23]
    std_image = np.std(group, axis=0)
    std_images.append(std_image)

# Stack the std images into a multi-dimensional array
stacked_std_images = np.stack(std_images)

# Set the output file name
output_file = 'stacked_std_images.npy'

# Save the stacked_median_images array as a numpy file
np.save(output_file, stacked_std_images)

### Normalization of  all images

In [ ]:
#empty list to store normalized images
normalized_images = []

#loop to normalize images
for i in range(25, len(images_array)):
    normalized_image=(images_array[i]-stacked_median_images[i-24])/stacked_std_images[i-24]
    normalized_images.append(normalized_image)
    
stacked_nor_images = np.stack(normalized_images)

# Define the output directory and file name template
#output_dir = 'X:\STUDENTS\GROUP_10\S1 images\normalizedVV\ '

#name of the images
n = 'normalized_'
Normalized_name = file_names[24:]

image_names =  []

for i, img in enumerate(stacked_nor_images):
    # Get the corresponding image name from the list
    img_name = n + Normalized_name[i]
    image_names.append(img_name)
print(image_names)

# Define some metadata for the output files

meta = {'driver': 'GTiff', 'dtype': 'float64', 'nodata': None, 'width': 1217, 'height': 697, 'count': 1, 'crs': 4326, 'transform': Affine(8.983152841195215e-05, 0.0, 110.16534402563957,
       0.0, -8.983152841195215e-05, -0.7573696160411686)}

# Loop over the images and file names in parallel, and save each image to disk
for img, fname in zip(stacked_nor_images, image_names):
    with rasterio.open(fname, 'w', **meta) as dst:
        dst.write(img, indexes=1)


## 3. Detection of change

In order to detect the change, we set a threshold to reclassify the image in a binary image. We try different thresholds between ... and ... For more accuracy we intersect VV and VH images and apply different filter (no filter, 3x3, 5x5, 7x7) to the intersection images.

### Threshold

In [ ]:
# threshold to detect deforestation
threshold = -0.5
cvt_threshold = str(threshold)


### Binary VH images

In [ ]:
# Set the directory path
dir_path_VH = 'X:/STUDENTS/GROUP_10/S1 images/normalizedVH'

# Get the list of file names in the directory
file_names_VH = os.listdir(dir_path_VH)

# Initialize an empty list to store the images
img_VH = []

# Loop over the file names and load each image into a NumPy array
for file_name in file_names_VH:
        # Load the image into a NumPy array
        with rasterio.open(os.path.join(dir_path_VH, file_name)) as dataset:
            src_VH = dataset.read(1)
        # Append the image to the list of images
        img_VH.append(src_VH)
        
#reclassifying the image
binary_VH = []

for src_VH in img_VH:
    binary_array_VH = np.where(src_VH <= threshold, 1, 0)
    binary_VH.append(binary_array_VH)

### Binary VV images

In [ ]:
# Set the directory path
dir_path_VV = 'X:/STUDENTS/GROUP_10/S1 images/normalizedVV'

# Get the list of file names in the directory
file_names_VV = os.listdir(dir_path_VV)

# Initialize an empty list to store the images
img_VV = []

# Loop over the file names and load each image into a NumPy array
for file_name in file_names_VV:
        # Load the image into a NumPy array
        with rasterio.open(os.path.join(dir_path_VV, file_name)) as dataset:
            src_VV = dataset.read(1)
        # Append the image to the list of images
        img_VV.append(src_VV)
        
#reclassifying the image
binary_VV = []

for src_VV in img_VV:
    binary_array_VV = np.where(src_VV <= threshold, 1, 0)
    binary_VV.append(binary_array_VV)

### Intersection of VV and VH

In [ ]:
#empty list to store images
intersections= []

#multiplication of the two binary image VV and VH

for VV, VH in zip(binary_VV, binary_VH):
    intersection= VV*VH
    intersections.append(intersection)

### Filter application

In [ ]:
#empty list to store images
intersections_f= []

#set of the filter size
filterwindow= 7
cvt_filterwindow= str(filterwindow)

#Filter application
for intersection in intersections:
    filtered_img= median_filter(intersection, size=filterwindow)
    intersections_f.append(filtered_img)


### Save each classification images

In [ ]:
#new empty list to store new names
intersection_names = []

#loop to create the new names

for file_name in file_names_VH:
    conserved_part = '_' + '_'.join(file_name.split('_')[4:7])
    intersection_name = 'intersection_'+ cvt_threshold +'_'+ cvt_filterwindow + conserved_part  
    intersection_names.append(intersection_name)
    
print(intersection_names)

# Define the output directory path
output_dir = r"X:\STUDENTS\GROUP_10\S1 images\test"

# Create the output directory if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define some metadata for the output files

meta = {'driver': 'GTiff', 'dtype': 'float64', 'nodata': None, 'width': 1217, 'height': 697, 'count': 1, 'crs': 4326, 'transform': Affine(8.983152841195215e-05, 0.0, 110.16534402563957,
       0.0, -8.983152841195215e-05, -0.7573696160411686)}

# Loop over the images and file names in parallel, and save each image to disk
for img, fname in zip(intersections_f, intersection_names):
    output_path = os.path.join(output_dir, fname )  # Construct the output file path
    with rasterio.open(output_path, 'w', **meta) as dst:
        dst.write(img, indexes=1)

## 4. Validation

For the validation, 20 polygons were drawn on QGIS based on an image of planet.com :  10 in the intact forest area and 10 in the deforested area.

In [ ]:
field_code = 'field'

#directory to the polygone layer
in_situ_val_shp ="X:/STUDENTS/GROUP_10/ColineDelmotte/poly/poly_for_nonFor20.shp"
in_situ_val_tif= "X:/STUDENTS/GROUP_10/validation_points/raster_validation.tif"

# Open the shapefile with GeoPandas
in_situ_gdf = gpd.read_file(in_situ_val_shp)

# Open the raster file you want to use as a template for rasterize
forest_tif = "X:/STUDENTS/GROUP_10/S1 images/test/intersection_-0.5_5_2021-08_cliped.tif"
src = rasterio.open(forest_tif, "r")

# Update metadata
out_meta = src.meta
out_meta.update(nodata=-900)

crs_shp = str(in_situ_gdf.crs).split(":",1)[1]
crs_tif = str(src.crs).split(":",1)[1]

print(f'The CRS of in situ data is    : {crs_shp}')
print(f'The CRS of raster template is : {crs_tif}')

### Rasterization of the polygon layer

In [ ]:
# Burn the features into the raster and write it out
dst = rasterio.open(in_situ_val_tif, 'w+', **out_meta)
dst_arr = dst.read(1)
    
# This is where we create a generator of geom, value pairs to use in rasterizing

geom_col = in_situ_gdf.geometry
code_col = in_situ_gdf[field_code].astype(int)

shapes = ((geom,value) for geom, value in zip(geom_col, code_col))

in_situ_arr = features.rasterize(shapes=shapes,
                                     fill=no_data,
                                     out=dst_arr,
                                     transform=dst.transform)

dst.write_band(1, in_situ_arr)

# Close rasterio objects
src.close()
dst.close()

### Build y_pred and y_true

In [ ]:
# Open in-situ used for validation
src = rasterio.open(in_situ_val_tif, "r")
val_arr = src.read(1)
src.close()

# Open classification map
src = rasterio.open(forest_tif, "r")
classif_arr = src.read(1)
src.close()

# Get the postion of validation pixels
idx = np.where(val_arr == -900, 0, 1).astype(bool)

# Ground truth (correct) target values
y_true = val_arr[idx]
print(len(y_true))

print(y_true)
print(f'Reference data (truth) : {y_true}')

# Estimated targets as returned by a classifier.
y_pred = classif_arr[idx]

print(f'Classification data    : {y_pred}')

In [ ]:
# Check if there are missing classes in the classification

classes_all  = sorted(np.unique(y_true))
classes_pred = sorted(np.unique(y_pred))

classes_missing = set(y_true) - set(y_pred)

print(f'{len(classes_missing)} classes are missing in the classification (y_pred) : {classes_missing} \n')

print(f'All training classes :\n {classes_all}')
print(f'All predicted classes (at least once) :\n {classes_pred}')

In [ ]:
classes_name = ('forest', 'non forest')

for code,name in zip(classes_all, classes_name):
    print(f'{code} - {name}')

### Accuracy metrics

In [ ]:
acc_metrics_str = classification_report(y_true,  # Exclure la première colonne de y_true
                                        y_pred,  # Exclure la première colonne de y_pred
                                        target_names=classes_name,
                                        labels=classes_all,
                                        digits=3,
                                        zero_division=0)  # Spécifier la valeur de division par zéro personnalisée

print(acc_metrics_str)

### Accuracy score

In [ ]:
oa = accuracy_score(y_true, y_pred)
oa = round(oa*100, 2)

print(f'Overall Accuracy : {oa}%')

## 5. Map

In [4]:
im_path = 'C:/Users/karma/Desktop/STUDY_UCL/Land_MonitoringEO/poster/GROUP_10/raster_processed/'

images = sorted(glob.glob(f'{im_path}intersection_-0.25_9*cliped.tif'), reverse = True)

#basename = os.basename(images[1])

new_tif = f'{im_path}CLASSIF_2017-2020_dis-sta___classified_with_combined_model_medianFilter3x3_ALL.tif'#_medianFilter5x5_ALL.tif'

nodata_val = -1000

ref_img = rasterio.open(images[1], "r", driver='GTiff')

profile = ref_img.profile
profile.update(
        dtype=rasterio.float32,  # Set to int16 it is lighter than float
        count=1,                 # We will write 1 band by file
        nodata=nodata_val,       # Set nodata value in metadata
        compress='lzw'           # Compression option)
)

new = np.full(ref_img.shape, nodata_val, dtype='float32')

ref_img.close()

for img in images:
    
    date_1 = os.path.basename(img)[21:25]
    date_2 = os.path.basename(img)[26:28]
    date = int(date_1 + date_2)
    
    # print(f'dates{date}')
    # print(f'dates{date_1}')
    # print(f'dates{date_2}')
    
    img = rasterio.open(img, "r", driver='GTiff')
    im = img.read(1)
    
   
    im = im.astype('float32')
    im = np.nan_to_num(im, nan=nodata_val)
    

    im[np.where(im == 0)] = 0
    im[np.where(im == 1)] = date

    print("Number of dates = {}".format(np.count_nonzero(im == date)))
    
    mask = np.where(im != 0)
    
    new[mask] = im[mask]


print(np.unique(new))

im_dst = rasterio.open(new_tif, "w", **profile)
im_dst.write(new, 1)
im_dst.close()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b812eaa8-a356-401a-a7e5-fb6c6f64af60' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>